In [2]:
# 如果你已經安裝過這些套件，可以略過這一格
%%bash
pip install --upgrade pip
pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo unsloth
pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer


  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 104.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 169.4 MB/s  0:00:00



In [3]:
from unsloth import FastLanguageModel
import torch

# 載入 4bit 版 Qwen3-14B
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name          = "unsloth/Qwen3-14B",
    max_seq_length      = 2048,
    load_in_4bit        = True,
    load_in_8bit        = False,
    full_finetuning     = False,  # 我們稍後用 PEFT
)

# 準備 LoRA Adapter
model = FastLanguageModel.get_peft_model(
    model,
    r                       = 32,
    target_modules         = ["q_proj","k_proj","v_proj","o_proj",
                              "gate_proj","up_proj","down_proj"],
    lora_alpha             = 32,
    lora_dropout           = 0,
    bias                   = "none",
    use_gradient_checkpointing = "unsloth",
    random_state           = 3407,
    use_rslora             = False,
    loftq_config           = None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.9: Fast Qwen3 patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.9.9 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [4]:
# STEP 0. 掛載 Google Drive
################################################################################
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from datasets import load_dataset, Dataset
import pandas as pd

# 1. 讀入你先前生成的 1286 筆 JSONL
raw = load_dataset(
    "json",
    data_files = {"/content/drive/MyDrive/sport/train_dataset/運動處方.json"},
    split      = "train"
)


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
def generate_conversation(examples):
    convs = []
    for inst, inp, out in zip(examples["instruction"],
                              examples["input"],
                              examples["output"]):
        # 把 None 變成空字串
        inst = inst or ""
        inp  = inp  or ""
        out  = out  or ""
        prompt = inst + (("\n" + inp) if inp else "")
        convs.append([
            {"role": "user",      "content": prompt},
            {"role": "assistant", "content": out},
        ])
    return {"conversations": convs}


In [10]:
# map 出「conversations」欄
convs = raw.map(
    generate_conversation,
    batched       = True,
    remove_columns= raw.column_names
)["conversations"]

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

In [12]:
from datasets import load_dataset, Dataset
import pandas as pd

# 1. 讀入您的 JSONL 資料
raw = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/sport/train_dataset/運動處方.json",
    split="train"
)

def generate_conversation(examples):
    convs = []
    for inst, inp, out in zip(examples["instruction"],
                              examples["input"],
                              examples["output"]):
        # 把 None 變成空字串
        inst = inst or ""
        inp = inp or ""
        out = out or ""
        prompt = inst + (("\n" + inp) if inp else "")
        convs.append([
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": out},
        ])
    return {"conversations": convs}

# 正確的處理方式
dataset = raw.map(
    generate_conversation,
    batched=True,
    remove_columns=raw.column_names
)

# 修正：正確取出 conversations
conversations_list = dataset["conversations"]

# 處理每個對話
formatted_texts = []
for conv in conversations_list:
    formatted_text = tokenizer.apply_chat_template(
        conv,  # 這裡傳入單一對話
        tokenize=False,
        add_generation_prompt=False,  # 訓練時通常設為 False
    )
    formatted_texts.append(formatted_text)

# 創建新的 dataset
final_dataset = Dataset.from_dict({"text": formatted_texts})

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

In [14]:
from datasets import load_dataset, Dataset
import pandas as pd

# 1. 讀入您的 JSONL 資料
raw = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/sport/train_dataset/運動處方.json",
    split="train"
)

def generate_conversation(examples):
    convs = []
    for inst, inp, out in zip(examples["instruction"],
                              examples["input"],
                              examples["output"]):
        # 把 None 變成空字串
        inst = inst or ""
        inp = inp or ""
        out = out or ""
        prompt = inst + (("\n" + inp) if inp else "")
        convs.append([
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": out},
        ])
    return {"conversations": convs}

# 處理資料
dataset = raw.map(
    generate_conversation,
    batched=True,
    remove_columns=raw.column_names
)

# 取出對話列表
conversations_list = dataset["conversations"]

# 格式化每個對話
formatted_texts = []
for conv in conversations_list:
    try:
        formatted_text = tokenizer.apply_chat_template(
            conv,
            tokenize=False,
            add_generation_prompt=False,
        )
        formatted_texts.append(formatted_text)
    except Exception as e:
        print(f"格式化對話時出錯: {e}")
        continue

# 創建最終的 Dataset
df = pd.DataFrame({"text": formatted_texts})
train_dataset = Dataset.from_pandas(df).shuffle(seed=3407)

print(f"成功處理 {len(formatted_texts)} 筆訓練資料")

Map:   0%|          | 0/1280 [00:00<?, ? examples/s]

成功處理 1280 筆訓練資料


In [15]:
# ── 安裝套件（如果已安裝可跳過） ─────────────────────────
# !pip install trl==0.15.2 unsloth bitsandbytes accelerate ...

# ── 匯入與自訂 Callback ────────────────────────────────
from trl import SFTTrainer, SFTConfig
from transformers import TrainerCallback

class SaveBestTrainLossCallback(TrainerCallback):
    def __init__(self):
        self.best_loss = float("inf")
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or "loss" not in logs:
            return control
        loss = logs["loss"]
        if loss < self.best_loss:
            self.best_loss = loss
            control.should_save = True
        else:
            control.should_save = False
        return control

# ── 其餘：載入模型、資料集等 ─────────────────────────────


In [16]:
from transformers import TrainerCallback
import os
import shutil

class SegmentedBestLossSaverCallback(TrainerCallback):
    def __init__(self, save_dir="checkpoints", segment_steps=50):
        self.segment_steps = segment_steps
        self.save_dir = save_dir
        self.best_loss = float("inf")
        self.best_step = None
        self.current_segment_start = 0
        self.best_model_dir = None

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or "loss" not in logs:
            return control

        step = state.global_step
        loss = logs["loss"]

        # 初次記錄
        if self.best_step is None:
            self.best_loss = loss
            self.best_step = step

        # 若在當前段落中發現更好的 loss，更新
        if self.current_segment_start < step <= self.current_segment_start + self.segment_steps:
            if loss < self.best_loss:
                self.best_loss = loss
                self.best_step = step
                control.should_save = True  # 將此步存下
        else:
            # 區間結束 ⇒ 複製 best checkpoint 到對應資料夾
            segment_end = self.current_segment_start + self.segment_steps
            segment_dir = os.path.join(self.save_dir, f"checkpoint-{segment_end}")
            source_dir = os.path.join(args.output_dir, f"checkpoint-{self.best_step}")

            if os.path.exists(source_dir):
                shutil.copytree(source_dir, segment_dir, dirs_exist_ok=True)
                print(f"✅ 存下區間 {self.current_segment_start+1}-{segment_end} 最佳 step {self.best_step}（loss={self.best_loss:.4f}）")

            # reset for next segment
            self.current_segment_start += self.segment_steps
            self.best_loss = float("inf")
            self.best_step = None
            control.should_save = False  # 避免重複儲存

        return control


In [ ]:
trainer = SFTTrainer(
    model         = model,
    tokenizer     = tokenizer,
    train_dataset = train_dataset,
    args = SFTConfig(
        dataset_text_field          = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps                   = 900,
        learning_rate               = 1e-4,
        logging_steps               = 1,
        optim                       = "adamw_8bit",
        save_strategy               = "no",     # 關掉內建 save，完全由 callback 控制
        save_total_limit            = 1000,     # 不限制（因為每段會手動 copy）
        lr_scheduler_type           = "linear",
        seed                        = 3407,
        report_to                   = "none",
        output_dir                  = "./trainer_output",  # 很重要！提供 checkpoint 存放來源
    ),
    callbacks=[
        SegmentedBestLossSaverCallback(
            save_dir="./segmented_checkpoints",  # 存到這個資料夾下
            segment_steps=50                    # 每 50 步一段
        )
    ],
)
trainer_stats = trainer.train()



Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/1280 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,280 | Num Epochs = 6 | Total steps = 900
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)


Step,Training Loss
1,5.558400
2,5.201300
3,5.411000
4,5.322900
5,5.054700
6,5.141600
7,5.085100
8,4.992500
9,4.715900
10,5.291300


Unsloth: Will smartly offload gradients to save VRAM!
✅ 存下區間 1-50 最佳 step 49（loss=1.1384）
✅ 存下區間 51-100 最佳 step 99（loss=0.7664）
✅ 存下區間 101-150 最佳 step 117（loss=0.7201）
✅ 存下區間 151-200 最佳 step 197（loss=0.4337）
✅ 存下區間 201-250 最佳 step 239（loss=0.4427）
✅ 存下區間 251-300 最佳 step 267（loss=0.3594）
✅ 存下區間 301-350 最佳 step 335（loss=0.2429）
✅ 存下區間 351-400 最佳 step 391（loss=0.3124）
✅ 存下區間 401-450 最佳 step 404（loss=0.2294）
✅ 存下區間 451-500 最佳 step 494（loss=0.1703）
✅ 存下區間 501-550 最佳 step 515（loss=0.1651）
✅ 存下區間 551-600 最佳 step 597（loss=0.1844）
✅ 存下區間 601-650 最佳 step 644（loss=0.1436）
✅ 存下區間 651-700 最佳 step 666（loss=0.1072）
✅ 存下區間 701-750 最佳 step 709（loss=0.1194）
✅ 存下區間 751-800 最佳 step 755（loss=0.1033）
✅ 存下區間 801-850 最佳 step 845（loss=0.0816）


In [20]:
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import TextStreamer

# ① 重新載入 4-bit Qwen3-14B
base, tok = FastLanguageModel.from_pretrained(
    "unsloth/Qwen3-14B",
    load_in_4bit=True,
    max_seq_length=2048,
)

# ② 指定本機最佳 LoRA
best_path = "/content/drive/MyDrive/sport/Qwen/trainer_output/checkpoint-870"
model = PeftModel.from_pretrained(base, best_path, is_trainable=False)

# ③ 切推論模式並確認 LoRA 掛載成功
model = FastLanguageModel.for_inference(model)
print(model.peft_config)                 # 應看到 r=32、lora_alpha=32…

# ④ 測試
messages = [{"role": "user", "content": "體適能定義是什麼"}]
prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tok(prompt, return_tensors="pt").to("cuda")

streamer = TextStreamer(tok, skip_prompt=True)
_ = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.8,
    streamer=streamer,
)


==((====))==  Unsloth 2025.9.9: Fast Qwen3 patching. Transformers: 4.56.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='unsloth/qwen3-14b-unsloth-bnb-4bit', revision=None, inference_mode=True, r=32, target_modules={'up_proj', 'o_proj', 'k_proj', 'v_proj', 'gate_proj', 'q_proj', 'down_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False, target_parameters=None)}
<think>

</think>

體適能是身體適應日常生活與環境變化的能力，尤其是適應運動的能力。<|im_end|>


In [ ]:
messages = [{"role": "user", "content": "請依下列健康數據擬定運動處方。年齡：66 歲，性別：男性，安靜心跳率：70 次/分，運動方式：游泳，運動目標：改善步態與下肢穩定性"}]
prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tok(prompt, return_tensors="pt").to("cuda")

streamer = TextStreamer(tok, skip_prompt=True)
_ = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.8,
    streamer=streamer,
)

<think>

</think>

最大心跳率 = 220 - 66 = 154
目標心跳率（40%HRR）= 0.4×(154-70)+70 ≈ 101
建議強度範圍：40～55%HRR

每週從事游泳 3～5 次，每次約 30～60 分鐘：
- 熱身：5 分鐘水中慢速滑行與肩部活動
- 主運動：間歇式主幹穩定與下肢推水動作，控制節奏避免速度過快
- 緩和：5 分鐘慢速滑行與呼吸調節
建議採用RPE 10～12區間監控強度，並觀察雙腿對稱性與平衡反應。

### 🗓️ 20 週運動處方表：

| 週數         | 運動頻率(次/週)    | 熱身(分)    | 主運動(分)    | 緩和(分)    | 總時間(分)     | 運動強度%HRR     |
| ------------ | ------------------ | ----------- | ------------- | ----------- | -------------- | ---------------- |
| 第 1～2 週   | 2                  | 5           | 20            | 5           | 30             | 40               |
| 第 3～4 週   | 3                  | 5           | 25            | 5           | 35             | 45               |
| 第 5～6 週   | 3                  | 5           | 30            | 5           | 40             | 45               |
| 第 7～8 週   | 3                  | 5           | 35            | 5           | 45             | 50               |
| 第 9～10 週  | 4                  | 5           | 40            | 5           | 

In [ ]:
!cp -r /content/trainer_output/checkpoint-861 /content/drive/MyDrive/sport/Owen/


cp: cannot stat '/content/trainer_output/checkpoint-861': No such file or directory


In [ ]:
# ④ 測試?
messages = [{"role": "user", "content": "什麼是身體活動"}]
prompt = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
inputs = tok(prompt, return_tensors="pt").to("cuda")

streamer = TextStreamer(tok, skip_prompt=True)
_ = model.generate(
    **inputs,
    max_new_tokens=1000,
    temperature=0.7,
    top_p=0.8,
    streamer=streamer,
)


<think>

</think>

身體肌肉收縮所產生的動作，範圍較廣，包括日常生活的活動與運動。<|im_end|>


In [ ]:
# =========================================
# 【Cell 6】另存 LoRA + tokenizer（最後一步）
# =========================================
model.save_pretrained("lora_adapter")
tok.save_pretrained("lora_adapter")
print("🚀 LoRA 與 tokenizer 已存到 ./lora_adapter/")


🚀 LoRA 與 tokenizer 已存到 ./lora_adapter/


In [ ]:
# =========================================
# 匯出 16-bit GGUF
# =========================================
gguf_dir = "qwen3_lora_f16"          # 目錄名稱可自行修改

model.save_pretrained_gguf(
    gguf_dir,
    tok,                              # tokenizer
    quantization_method="f16"         # ★ 16-bit 半精度
)

print(f"✅ 已匯出 16-bit GGUF 到 ./{gguf_dir}/")


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 11.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 61.52 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 85%|████████▌ | 34/40 [00:00<00:00, 48.01it/s]
We will save to Disk and not RAM now.
100%|██████████| 40/40 [00:05<00:00,  7.80it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting qwen3 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at qwen3_lora_f16 into f16 GGUF format.
The output location will be /content/qwen3_lora_f16/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: qwen3_lora_f16
INFO:hf-to-gguf:Model architecture: Qwen3ForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00006.safetensors'
IN

In [ ]:
!pip install llama-cpp-python --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 102.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4123153 sha256=16e9e7233ecb2a9c2ba1b17e54a154987ab29481c8535cc1ac62a4c50cc1be04
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [llama-cpp-python]


In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/qwen3_lora_q4/unsloth.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=4,     # 根據 Colab CPU 配置
    use_mlock=False,
)


ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
prompt = """你是一位運動科學專家。

請根據運動科學教材，回答以下問題，僅需回答相對應的內容，可以很豐富，但不需補充其他問題或延伸內容。

Q: 請依據以下身體數據制定個人化運動處方。",
    "input": "年齡：26 歲\n性別：男性\n安靜心跳率：75 次/分\n運動目標：提升柔軟度與姿勢控制
A:"""

output = llm(
    prompt,
    max_tokens=1000, # 防止進入補問模式
)

print(output["choices"][0]["text"].strip())


Llama.generate: 100 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     447.62 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =  317005.63 ms /  1000 runs   (  317.01 ms per token,     3.15 tokens per second)
llama_perf_context_print:       total time =  319364.60 ms /  1001 tokens


根據身體數據制定個人化運動處方：
年齡：26 歲（年輕族群）
性別：男性
安靜心跳率：75 次/分（正常範圍）
運動目標：提升柔軟度與姿勢控制
建議運動類型：瑜珈、太極、身體重組與功能性訓練
運動頻率：每週 3～5 次
每次運動時間：30～60 分鐘
運動強度：中等強度（最大心跳率 60～70%）
心率監測：建議配戴心率帶或使用運動手錶
運動計畫範例：
第1週：每次運動 30 分鐘，以熱身與輕度活動開始
第2週：增加至 45 分鐘，加入瑜珈與姿勢訓練
第3週：運動時間延長至 60 分鐘，加入功能性訓練與有氧活動
第4週：根據身體適應調整強度與持續時間，每週漸進增加運動量
建議搭配飲食與休息：運動後補充足夠水分與蛋白質，確保每晚有 7～9 小時睡眠，並每週安排 1～2 天休息日避免過度訓練。

### 🗓️ 20週運動處方表：
| 週數 | 運動類型       | 運動時間（分） | 運動頻率（次/週） | 運動強度（%最大心跳率） |
|------|----------------|----------------|--------------------|---------------------------|
| 第1週 | 熱身與輕度活動 | 30             | 3                  | 50                        |
| 第2週 | 瑜珈與姿勢訓練 | 45             | 4                  | 55                        |
| 第3週 | 功能性訓練     | 60             | 5                  | 60                        |
| 第4週 | 綜合訓練       | 75             | 6                  | 65                        |
| 第5週 | 高強度間歇訓練 | 90             | 7                  | 70                        |
| 第6週 | 瑜珈與伸展     | 60             | 5                  | 

In [ ]:
!ls -lh /content/qwen3_lora_f16
!stat /content/qwen3_lora_f16/unsloth.F16.gguf


total 56G
-rw-r--r-- 1 root root  707 May 19 04:15 added_tokens.json
-rw-r--r-- 1 root root  787 May 19 04:15 config.json
-rw-r--r-- 1 root root  237 May 19 04:15 generation_config.json
-rw-r--r-- 1 root root 1.6M May 19 04:15 merges.txt
-rw-r--r-- 1 root root 4.7G May 19 04:15 model-00001-of-00006.safetensors
-rw-r--r-- 1 root root 4.7G May 19 04:16 model-00002-of-00006.safetensors
-rw-r--r-- 1 root root 4.6G May 19 04:16 model-00003-of-00006.safetensors
-rw-r--r-- 1 root root 4.7G May 19 04:16 model-00004-of-00006.safetensors
-rw-r--r-- 1 root root 4.6G May 19 04:17 model-00005-of-00006.safetensors
-rw-r--r-- 1 root root 4.5G May 19 04:17 model-00006-of-00006.safetensors
-rw-r--r-- 1 root root  36K May 19 04:17 model.safetensors.index.json
-rw-r--r-- 1 root root  614 May 19 04:15 special_tokens_map.json
-rw-r--r-- 1 root root  11K May 19 04:15 tokenizer_config.json
-rw-r--r-- 1 root root  11M May 19 04:15 tokenizer.json
-rw-r--r-- 1 root root  28G May 19 04:19 unsloth.F16.gguf
-rw-r-

In [ ]:


# 或直接匯出為 4-bit GGUF
model.save_pretrained_gguf(
    "qwen3_lora_q4",
    tokenizer,
    quantization_method="q4_k_m"
)
print("已匯出 4-bit GGUF 模型到 ./qwen3_lora_q4/")


In [ ]:
# STEP  掛載 Google Drive
################################################################################
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/trainer_output /content/drive/MyDrive/sport/Owen/


In [ ]:
from pathlib import Path
import shutil

# 來源檔（只複製 Q4_K_M）
src_file = Path("/content/qwen3_lora_q4/unsloth.Q4_K_M.gguf")

# 目標資料夾
dst_dir = Path("/content/drive/MyDrive/sport/Llama3/gguf_only")
dst_dir.mkdir(parents=True, exist_ok=True)

# 複製檔案
shutil.copy(str(src_file), str(dst_dir / src_file.name))

print(f"✅ 已複製 {src_file.name} 到 {dst_dir}")


✅ 已複製 unsloth.Q4_K_M.gguf 到 /content/drive/MyDrive/sport/Llama3/gguf_only
